In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/FairPrune"

/content/drive/MyDrive/FairPrune


In [ ]:
%cd "/content/drive/MyDrive/UNSW/FairPrune"


/content/drive/.shortcut-targets-by-id/1mE95vsJWdCLeyosiSlxTV5_-JveJMu0s/FairPrune


In [12]:
!python Base.py --config Fitz17k_configs.yml

CUDA is available: True 

Starting... 

{'train': 12809, 'val': 3203}
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Fitz17kResNet18(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=

In [3]:
!pip install -q backpack-for-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00


In [9]:
!python FairPrune.py --config Fitz17k_configs.yml

CUDA is available: True 

Starting... 

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loading model from: /content/drive/MyDrive/FairPrune/Outputs/Resnet18_checkpoint_BASE.pth
+++++++++++++++++++++++++++++ Pruning Iteration 0 +++++++++++++++++++++++++++++
{'train': 8854, 'val': 2209}
{'train': 3955, 'val': 994}
processing batch 0
processing batch 1
processing batch 2
processing batch 3
processing batch 4
processing batc